<a href="https://colab.research.google.com/github/myazann/Text-Generation/blob/main/Generate_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets==1.0.2

!pip install git-python==1.0.3
! pip install tokenizers

import os
import numpy as np
from transformers import OpenAIGPTLMHeadModel, Trainer, TrainingArguments, GPT2LMHeadModel
from transformers import AutoModel, AutoModelWithLMHead, AutoTokenizer, EncoderDecoderModel, BertTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling,LineByLineTextDataset
from transformers import pipeline
import torch
import gc
import datasets
from dataclasses import dataclass, field
from typing import Optional
from tokenizers import BertWordPieceTokenizer, Tokenizer
from tokenizers.processors import BertProcessing

Loading data, tokenizer and the model.


In [10]:
if os.listdir("/content/drive/My Drive/enc2dec"):
  last_trained_path = "/content/drive/My Drive/enc2dec" + "/" + os.listdir("/content/drive/My Drive/enc2dec")[0]
else:
  last_trained_path = None 

tg_data_train = datasets.load_dataset("csv", data_files = "/content/drive/My Drive/telegram_chatbot_train.csv", split = "train")
tg_data_val = datasets.load_dataset("csv", data_files = "/content/drive/My Drive/telegram_chatbot_val.csv", split = "train")


tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased', do_lower_case=True, return_special_tokens_mask=True)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

model = EncoderDecoderModel.from_encoder_decoder_pretrained("dbmdz/bert-base-turkish-cased", "dbmdz/bert-base-turkish-cased")

if last_trained_path is not None:
  model = model.from_pretrained(last_trained_path).cuda()

Using custom data configuration default
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-52f8e7c0f5270178/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277)
Using custom data configuration default
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-36c2692e7da1159c/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277)
Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

Sample code to generate text given an input. Different decoding strategies can be used.

In [ ]:
start = tokenizer("aykut mert", return_tensors="pt").input_ids
out = model.generate(start.cuda(), min_length = 5, max_length = 150, 
                     
                    ## top k-sampling
                    ##top_k = 50, do_sample = True  
                     
                    ## top p-sampling
                    ##top_p = 0.95, do_sample = True 

                    #beam search
                    no_repeat_ngram_size = 3, early_stopping=True, num_beams = 4, length_penalty = 2.0,

                    temperature=0.95,
                    ##num_return_sequences = 3
                     )

decoded = tokenizer.batch_decode(out)[0].split()
kisi = decoded[1]
msg = decoded[2:-1]

kisi + ":" + " ".join(([str(elem) for elem in msg]))

Start a conversation. 

In [ ]:
i = 0
start_sentence = input()

while i < 25:

  start = tokenizer(start_sentence, return_tensors="pt").input_ids
  out = model.generate(start.cuda(),  min_length = 5, max_length = 150,
                      no_repeat_ngram_size = 3, early_stopping=True, num_beams = 4, length_penalty = 2.0,
                      temperature=0.75)
  

  decoded = tokenizer.batch_decode(out)[0]

  kisi = decoded.split()[1]
  msg = decoded.split()[2:-1]

  print(kisi + ":" + " ".join(([str(elem) for elem in msg])))
  start_sentence = decoded

  i += 1